In [1]:
#https://www.tensorflow.org/tutorials/quickstart/advanced 

import tensorflow as tf

print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model

import tensorflow.keras as keras

import numpy as np

import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K

2024-08-08 00:34:54.664565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.10.0


In [2]:
# Universal parameters
batchSize = 1100 # batchSize = number of paths to be generated

d = 2 # dimension of forward process

batchSize_DNN = 1 #2 #10 # 20 50
Max_EPOCHS = 600
Mid_EPOCHS = 300#200
Min_EPOCHS = 300#100#60
lr = .001
seed = 0 #168
tf.random.set_seed(
    seed
)
tf.keras.utils.set_random_seed(
    seed
)

print('Number of paths (sample data) = ', batchSize)
print('batchSize = ', batchSize_DNN)
print('NumEpochs = ', Max_EPOCHS, Mid_EPOCHS, Min_EPOCHS)
print('Learning Rate = ', lr)



X_0 = 100 # initial price
print('initial price = ', X_0) 
V_0 = .15  # initial volatility
print('initial Vol = ', V_0)
K = 95 #90 #85 #80 


mu = .08
print('mu = ', mu )
alpha = 5 
print('alpha = ', alpha)
m = 0 
print('m = ', m)
nu = 1
print('nu = ', nu)
a = nu * np.sqrt(2 * alpha)
print('nu * sqrt(2 alpha ) = ', a)
rho = -.2
print('rho = ', rho)
gamma = 1
print('gamma = ', gamma )
eta = .2 
print('eta = ', eta)
timeSteps = 10  #20 #50 
print('Total number of time steps = ', timeSteps )
r = 0.02 # .05
print('risk free rate r = ', r)
T = 1/4 # terminal time
print('T = ', T)
dt = T / timeSteps
print('dt = ', dt)

strikes = [70,75,80,85,90,95,100,105,110,115]

# strikes = [i for i in range(70,116,2)]
#print('strikes = ', strikes)
#print('logMoney = ', np.array(strikes) / X_0)
# for loop here 
#ivValues = []
print('-------------Beginning computation-----------------')




Number of paths (sample data) =  1100
batchSize =  1
NumEpochs =  600 300 300
Learning Rate =  0.001
initial price =  100
initial Vol =  0.15
mu =  0.08
alpha =  5
m =  0
nu =  1
nu * sqrt(2 alpha ) =  3.1622776601683795
rho =  -0.2
gamma =  1
eta =  0.2
Total number of time steps =  10
risk free rate r =  0.02
T =  0.25
dt =  0.025
-------------Beginning computation-----------------


In [3]:
# First step is to train the deep neural network y^_{N-1} (a function of X_{N-1}, dW_{N-1}) by:
# input: X_{N-1}, dW_{N-1}
# output: y^_{N} = F(t_{N-1}, X_{N-1}, y^_{N-1}, z_{N-1}, dW_{N-1}) 
# (a custom layer is needed for a F transformation)
# minimizing quadratic loss: E( (y_{N} - y^_{N})^2 ), expectation is taken over all paths
# the best model y*_{N} = best(y^_{N}) is set to be the target of next deep neural network

In [4]:
def trainingData(seed): # = 168):
    tf.random.set_seed(seed)
    std_normal = tf.random.normal([batchSize,  # [x,   [number of paths,
                                   timeSteps + 1,  #  y, =  number of values in each path,            = tensor shape
                                   2],         #  z]    dim of each path 
                                  mean = 0.0,
                                  stddev = 1.0,
                                  dtype = tf.dtypes.float32,
                                  seed = seed,
                                  name = None)
    dW = std_normal * np.sqrt(dt)
    X = tf.Variable(tf.ones([batchSize, timeSteps + 1, 2], 
                            dtype = tf.dtypes.float32) * tf.constant([X_0, V_0]))
    for i in range(timeSteps):
        stockTerm1 = mu * X[:,i, 0] * dt
        stockTerm2 = (.7 / (np.pi) * (tf.math.atan(X[:,i, 1] - 1) + np.pi/2) + .03) * X[:, i,0] * dW[:,i,0] # sigma(Y_t)
        stockTerm3 = -r * X[:,i,0] * dt
        X[:, i+1, 0].assign(X[:,i, 0] + stockTerm1 + stockTerm2 + stockTerm3)
        volTerm1 = alpha * (m - X[:,i,1]) * dt
        volTerm2 = a * (rho *dW[:,i,0] + np.sqrt(1 - rho ** 2)* dW[:,i,1])
        X[:, i+1, 1].assign(X[:, i, 1] + volTerm1 + volTerm2) 
        
       # X = X[:, i+1, :].assign(X[:, i, :] + r*X[:, i, :]*dt + sigma*X[:, i, :]*dW[:, i, :])
    return X, dW

    

#terminal condition
def fun_g(X, K, t):
    # print(X.shape)
    g = tf.math.maximum(K * tf.math.exp((-r * (t/timeSteps) * T)) - X, 0)
    return g



 #X2=X[:,t, 1]
# driver 1    
def fun_f(t, X1, X2, z1, z2):
    sigma = ((.7 / (np.pi)) * (tf.math.atan(X2 - 1) + np.pi/2) + .03)
    term1 = -1 * z1 * ((mu - r) / sigma) 
    term2 = -1 * (1/(2 * gamma)) * ((-(mu - r) / sigma) **2)
    term3 =  (1/(2 * gamma)) * (gamma / 2) * z2 ** 2
    term4 = -1 * (1/(2 * gamma)) * eta * ((mu - r) / sigma) * z2

    regular = term1 + term2 + term3 + term4
    
    return regular


# function F (the estimation of y_{i+1} given t_i, X_i, y_i, z_i, dW_i)
def fun_F(t, X1, X2, y, z1, z2, dW1, dW2):
    F = y - fun_f(t, X1, X2, z1, z2) * dt + z1 * dW1 + z2 * dW2
    return F





In [5]:
X, dW = trainingData(seed)
print('sample path = ', X[1,:,:])
# print('sample brownian motion = ', dW[0,:,:])
print('sample BM mean: ', tf.math.reduce_mean(dW), '| variance: ', tf.math.reduce_variance(dW))
                    

sample path =  tf.Tensor(
[[ 1.00000000e+02  1.50000006e-01]
 [ 1.00554573e+02 -2.82164752e-01]
 [ 1.02307930e+02 -1.57119930e-02]
 [ 9.80057449e+01 -2.48888656e-01]
 [ 1.04667213e+02 -3.32593203e-01]
 [ 1.06612518e+02 -3.46326292e-01]
 [ 1.06523796e+02 -4.39549088e-02]
 [ 1.08641037e+02 -5.97284615e-01]
 [ 1.10312263e+02  2.68377781e-01]
 [ 1.19024925e+02  5.41483462e-01]
 [ 1.06819839e+02  5.83199024e-01]], shape=(11, 2), dtype=float32)
sample BM mean:  tf.Tensor(0.001961416, shape=(), dtype=float32) | variance:  tf.Tensor(0.024696153, shape=(), dtype=float32)


2024-08-08 00:34:59.971574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = Flatten()
        
        self.d1_y = keras.layers.Dense(10+d, activation='relu', input_shape=(2,))
        self.d2_y = keras.layers.Dense(10+d, activation='relu')
        self.d3_y = keras.layers.Dense(1)
        
        self.d4_z = keras.layers.Dense(10+d, activation='relu', input_shape=(2,))
        self.d5_z = keras.layers.Dense(10+d, activation='relu')
        self.d6_z = keras.layers.Dense(2)
        
    def call(self, y):
        
        y_0 = tf.reshape(y[:,0], (batchSize_DNN,1), name=None) ## X
        y_1 = tf.reshape(y[:,1], (batchSize_DNN,1), name=None) ## V
        y_2 = tf.reshape(y[:,2], (batchSize_DNN,1), name=None) ## dW1
        y_3 = tf.reshape(y[:,3], (batchSize_DNN,1), name=None) ## dW2
        
        y_raw = tf.concat([y_0,y_1], 1)
        
        y = self.flatten(y_raw)
        y = self.d1_y(y)
        y = self.d2_y(y)
        y = self.d3_y(y)
        
        y_export = y
        
        z = self.flatten(y_raw)
        z = self.d4_z(z)
        z = self.d5_z(z)
        z = self.d6_z(z)
        
        z_export = z
        z_1 = z[:,0]
        z_2 = z[:,1]
       
        p = fun_F(t = time, # not useful
                  X1 = y_0,
                  X2 = y_1,
                  y = y,
                  z1 = z_1,
                  z2 = z_2,
                  dW1 = y_2,
                  dW2 = y_3)
             # t is a placeholder only, not useful, 
             # X is the first column of input data (input data = (X,dW))
             # y is the output of the last dense layer (1st dim of output (Y,z))
             # z is the sigma^T*D_x*y, where D_x*y is the numerical differentiation of y
             # dW is the second column of input data (input data = (X,dW))

        return p, y_export, z_export # dimensions (1, 1, 2)

In [7]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [8]:
train_MSE = tf.keras.metrics.MeanSquaredError(name='train_MSE')
test_MSE = tf.keras.metrics.MeanSquaredError(name='test_MSE')

In [9]:
@tf.function
def train_step(x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training = True)[0]
        loss = loss_object(y_train, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_MSE(y_train, predictions)

In [10]:
@tf.function
def test_step(x_test, y_test):
    predictions = model(x_test, training = False)[0]
    #t_loss = loss_object(y_test, predictions)

    test_MSE(y_test, predictions)

In [11]:
X, dW = trainingData(seed)

In [12]:
model = MyModel() 

X, dW = trainingData(seed)
#X1 = X[:,timeSteps, 0]

print("Strike = ", K)

#Test_MSE_list = []
   #DO TERMINAL TIME SEPARATELY 
for time in range(timeSteps-1, -1, -1):
    #model.set_weights(initial_weights)
    
   

    time_plus1 = time + 1
    path_ds = tf.concat([X, dW], axis = 2)
    path_ds = path_ds[:,time,:]
    #print("input data shape:")
    #print(path_ds.shape)
    #p = path_ds[:,0]#
    
    #print(p.shape)

    if time == timeSteps-1:
        payoff_timeplus1 = fun_g(X[:,time_plus1,0], K = K, t = time_plus1)#
    else :
        payoff_timeplus1 = np.loadtxt("RBSDE_predictions-Seller_"+str(time_plus1)+"K_"+str(K)+".csv", delimiter = ",", dtype = 'float32')
        payoff_timeplus1 = payoff_timeplus1.reshape((1100, 1))
     
    #print("output data shape:")
    #print(payoff_timeplus1.shape)
    
    
    (x_train, y_train), (x_test, y_test) = (path_ds[0:1000,:], 
                       payoff_timeplus1[0:1000]), (path_ds[1000:1100,:], 
                                                payoff_timeplus1[1000:1100])
    #print("input training data shape:")
    #print(x_train.shape)
    #print("input test data shape:")
    #print(x_test.shape)
    #print("output training data shape:")
    #print(y_train.shape)
    #print("output test data shape:")
    #print(y_test.shape)
    

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(batchSize_DNN)

    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batchSize_DNN)#.shuffle(100).batch(batchSize_DNN)

    if time == timeSteps-1:
        EPOCHS = Max_EPOCHS
    elif time < 6:
        EPOCHS = Mid_EPOCHS
    else :
        EPOCHS = Min_EPOCHS    
    
    i=1
    for epoch in range(EPOCHS):
        if i < 1 and epoch > EPOCHS:
            break
            
        train_MSE.reset_states()
        test_MSE.reset_states()
        
        for x_train, y_train in train_ds:
            train_step(x_train, y_train)
        
        for x_test, y_test in test_ds:
            test_step(x_test, y_test)
        
        if (epoch + 1) %100 == 0 :# and verbose == True:
            print(
            f'Epoch {epoch + 1}, '
            f'MSE: {train_MSE.result() * 100}, '
            f'Test MSE: {test_MSE.result() * 100}'
               )
        i = train_MSE.result()

    predictions = model.predict(path_ds, batch_size = batchSize_DNN)
    predictions_time = predictions[0].reshape((batchSize,1))
    Boundary = np.loadtxt("Boundarypredictions-Seller_"+str(time)+"K_"+str(K)+".csv", delimiter = ",") 
    Boundary = Boundary.reshape((batchSize,1))
    predictions_time_y = tf.math.maximum(predictions[1], Boundary)
    #Test_MSE_list.append(test_MSE(payoff_timeplus1, predictions_time))

    print("Time = ", time)
    print(test_MSE(payoff_timeplus1, predictions_time))

    print(np.mean(payoff_timeplus1))
    print(np.mean(predictions_time))
    print(np.mean(predictions_time_y))
    
    
    np.savetxt("RBSDE_predictions-Seller_"+str(time)+"K_"+str(K)+".csv", predictions_time_y, delimiter = ",")

    model.reset_states()
    
#np.savetxt("Test_MSE_for_Stike_"+str(K)+".csv", Test_MSE_list, delimiter = ",")    
   

Strike =  95
Epoch 100, MSE: 325.03057861328125, Test MSE: 216.1601104736328
Epoch 200, MSE: 169.00498962402344, Test MSE: 186.0557403564453
Epoch 300, MSE: 123.0693359375, Test MSE: 81.35115814208984
Epoch 400, MSE: 116.92938995361328, Test MSE: 72.00753784179688
Epoch 500, MSE: 108.87716674804688, Test MSE: 85.17510986328125
Epoch 600, MSE: 85.7653579711914, Test MSE: 76.48466491699219
1100/1100 [==============================] - 1s 765us/step
Time =  9
tf.Tensor(0.7645236, shape=(), dtype=float32)
2.2065997
2.4342475
2.5470042
Epoch 100, MSE: 67.94859313964844, Test MSE: 59.27504348754883
Epoch 200, MSE: 57.098812103271484, Test MSE: 55.76038360595703
Epoch 300, MSE: 53.93305969238281, Test MSE: 37.914398193359375
1100/1100 [==============================] - 1s 575us/step
Time =  8
tf.Tensor(0.38171792, shape=(), dtype=float32)
2.5470042
2.6100929
2.5957932
Epoch 100, MSE: 60.93193435668945, Test MSE: 46.04468536376953
Epoch 200, MSE: 54.142154693603516, Test MSE: 63.373409271240234

In [13]:
print(np.mean(predictions_time_y))

2.7576926


In [14]:
RBSDE = np.loadtxt("RBSDE_predictions-Seller_"+str(0)+"K_"+str(K)+".csv", delimiter = ",", dtype = 'float32')
BSDE0payoff = np.loadtxt("BSDE0predictions_"+str(0)+"_y.csv", delimiter = ",") 
Price = RBSDE - BSDE0payoff
print('Seller Price = ', np.mean(Price))
print('variance = ', np.var(Price))


Seller Price =  2.7694418132305145
variance =  0.0


In [15]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  36        
                                                                 
 dense_1 (Dense)             multiple                  156       
                                                                 
 dense_2 (Dense)             multiple                  13        
                                                                 
 dense_3 (Dense)             multiple                  36        
                                                                 
 dense_4 (Dense)             multiple                  156       
                                                                 
 dense_5 (Dense)             multiple                  26 

In [17]:
#implied volatility
def d1(sigma, t, T, S, K, r):
    frac = 1 / (sigma * np.sqrt(T - t))
    secondfactor = np.log(S / K) + (r + ((sigma ** 2) / 2)) * (T - t)
    return frac * secondfactor

def d2(sigma, t, T, S, K, r):
    return d1(sigma, t, T, S, K, r) - (sigma * np.sqrt(T - t))

def put_price(sigma, t, T, S, K, r):
    return (norm.cdf(-d2(sigma, t, T, S, K, r)) * K * np.exp(-r * (T - t))) - (norm.cdf(-d1(sigma, t, T, S, K, r)) * S)

def call_price(sigma, t, T, S, K, r):
    return S*norm.cdf(d1(sigma, t, T, S, K,r)) - K*np.exp(-r*(T - t)*norm.cdf(d2(sigma, t, T, S, K, r)))
def implied_vol(price, t, T, S, K, r):
    sigma1 = 0.1
    sigma2 = 0.7
    value1 = put_price(sigma1, t, T, S, K, r) - price
    value2 = put_price(sigma2, t, T, S, K, r) - price
    
    while value1 * value2 > 0:
        sigma1 = sigma1 - 0.01
        sigma2 = sigma2 + 0.01
        value1 = put_price(sigma1, t, T, S, K, r) - price
        value2 = put_price(sigma2, t, T, S, K, r) - price
    i = 0
    while True:
        i = i + 1
        if value1 * value2 == 0:
            if value1 == 0:
                imp_vol = sigma1 #print('implied volatility=', sigma1)
            elif value2 == 0:
                imp_vol = sigma2 #print('implied volatility=', sigma2)
            break
        elif value1 * value2 < 0:
            midpoint = (sigma1 + sigma2) / 2
            if (put_price(midpoint, t, T, S, K, r) - price) * value1 < 0:
                sigma1 = sigma1
                sigma2 = midpoint
                value1 = put_price(sigma1, t, T, S, K, r) - price
                value2 = put_price(sigma2, t, T, S, K, r) - price

            elif (put_price(midpoint, t, T, S, K, r) - price) * value2 < 0:
                sigma1 = midpoint
                sigma2 = sigma2
                value1 = put_price(sigma1, t, T, S, K, r) - price
                value2 = put_price(sigma2, t, T, S, K, r) - price

        if abs(sigma1 - sigma2) < 0.000001:
            imp_vol = sigma2
            break
    return imp_vol




In [18]:
strikes = [80, 85, 90, 95, 100, 105, 110, 115]
Price_list = []
Strike_list = []
for K in strikes:
    RBSDE = np.loadtxt("RBSDE_predictions-Seller_"+str(0)+"K_"+str(K)+".csv", delimiter = ",", dtype = 'float32') 
    BSDE0payoff = np.loadtxt("BSDE0predictions_"+str(0)+"_y.csv", delimiter = ",") 
    Price = RBSDE - BSDE0payoff
    print('Strike =', K, 'Buyer Price = ', np.mean(Price))
    Strike_list.append(K)
    Price_list.append(np.mean(Price))

np.savetxt("Amer_Seller_prices.csv", Price_list, delimiter = ",") 
Seller_price = np.array(Price_list)
Strike_array = np.array(Strike_list)
# plotting the points 
plt.plot(Strike_array, Buyer_price)
  
# naming the x axis
plt.xlabel('Strikes')
# naming the y axis
plt.ylabel('Price')
  
# giving a title to my graph
plt.title('Risk Indiffernce prices of Seller')
  
# function to show the plot
plt.show()




Strike = 80 Buyer Price =  0.3837037980556488
Strike = 85 Buyer Price =  0.6632647216320038
Strike = 90 Buyer Price =  1.1494762599468231
Strike = 95 Buyer Price =  2.7694418132305145
Strike = 100 Buyer Price =  4.522848337888718
Strike = 105 Buyer Price =  7.60872957110405
Strike = 110 Buyer Price =  12.546031683683395
Strike = 115 Buyer Price =  18.280828207731247


NameError: name 'Buyer_price' is not defined